In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

data = pd.read_csv("bank-additional-full.csv")

# Replacing missing values
data["marital"].replace("unknown", "married", inplace=True)
data["default"].replace("unknown", "no", inplace=True)
data["loan"].replace("unknown", "no", inplace=True)

# Dropping remaining missing values & outliers
data.drop(data[data.housing == "unknown"].index, inplace=True)
data.drop(data[data.education == "unknown"].index, inplace=True)
data.drop(data[data.job == "unknown"].index, inplace=True)
data.drop(data[data.age >= 70].index, inplace=True)

X = data.drop(columns=["y", "duration"])
y = data.y

X.describe(include="all")

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,37899.000000,37899,37899,37899,37899,37899,37899,37899,37899,37899,37899.000000,37899.000000,37899.000000,37899,37899.000000,37899.000000,37899.000000,37899.000000,37899.000000
unique,NaN,11,3,7,2,2,2,2,10,5,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN
freq,NaN,9943,22974,11787,37896,20376,31991,24120,12793,7943,NaN,NaN,NaN,32879,NaN,NaN,NaN,NaN,NaN
mean,39.458825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.574158,965.357661,0.164859,NaN,0.108509,93.574986,-40.585190,3.651885,5168.879720
std,9.574816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.778507,179.656227,0.477472,NaN,1.551501,0.571991,4.566371,1.716182,70.583983
min,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.354000,5099.100000
50%,38.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,999.000000,0.000000,NaN,1.100000,93.444000,-41.800000,4.857000,5191.000000
75%,46.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000


In [12]:
# Encoding categorical attributes
encoder = LabelEncoder()

label_enc_X = X
label_enc_X.job = encoder.fit_transform(label_enc_X.job)
label_enc_X.marital = encoder.fit_transform(label_enc_X.marital)
label_enc_X.education = encoder.fit_transform(label_enc_X.education)
label_enc_X.default = encoder.fit_transform(label_enc_X.default)
label_enc_X.housing = encoder.fit_transform(label_enc_X.housing)
label_enc_X.loan = encoder.fit_transform(label_enc_X.loan)
label_enc_X.contact = encoder.fit_transform(label_enc_X.contact)
label_enc_X.month = encoder.fit_transform(label_enc_X.month)
label_enc_X.day_of_week = encoder.fit_transform(label_enc_X.day_of_week)
label_enc_X.poutcome = encoder.fit_transform(label_enc_X.poutcome)

label_enc_model = SVC(random_state=42)

label_scores = cross_val_score(label_enc_model, label_enc_X,y, cv=10, n_jobs=-1, error_score='raise')
label_scores.mean()

0.9008406016304662

In [ ]:
one_enc_X = pd.get_dummies(data.drop(columns=["y", "duration"]))

one_enc_model = SVC(random_state=42, probability=True)

scores = cross_val_score(one_enc_model, one_enc_X,y, cv=10, n_jobs=-1, error_score='raise')
scores.mean()

In [6]:
scores_rms = cross_val_score(one_enc_model, one_enc_X, y, cv=10, n_jobs=-1, scoring='neg_brier_score', error_score='raise')
scores_rms.mean()

-0.08673736095152965

In [ ]:
# Using ROC one hot encoded

scores_roc = cross_val_score(one_enc_model, one_enc_X,y, n_jobs=-1, scoring='roc_auc')

scores_roc.mean()